- Title: OpenEphys on the Raspberry Pi 4
- Date: 2019-09-21 10:01
- Category: neuroscience
- Tags: raspberry pi, open ephys, neuroscience, neural recordings, open science
- Slug: open-ephys-raspberry-pi-4
- Authors: Tim Sainburg
- githuburl: 
- Summary: Instructions for how I set up Open Ephys on the Raspberry Pi 4 Model B

This summer Raspberry Pi released their new [Raspberry Pi 4 Model B](https://www.raspberrypi.org/products/raspberry-pi-4-model-b/) which features a number of new improvements, including two USB 3.0 ports, processors that tout ~3x performance increase over the last model, and up to 4GB ram. 

<div style="text-align:center; clear:both">
      <img src="assets/img/rpi4.png" style="display:inline-block">

<p style="text-align:center;font-style:italic">Updates for the raspberry pi 4</p> 
</div>

At the same time, I've been setting up a new rig to run chronic electrophysiology experiments in [our lab](http://gentnerlab.ucsd.edu/), using the OpenEphys open source neural acquisition board, while the behaviors for the experiment are all running off of the [RPiOperant board](https://github.com/theilmbh/RPiOperant) we designed in lab over the past two years (primarily designed by [Brad Theilman](http://bradtheilman.com/)):

<div style="text-align:center; clear:both; width:100%">
      <img src="assets/img/open-ephys-board.jpg", width="60%" style="display:inline-block; float:left">
      <img src="assets/img/magpiboard.png", width="40%" style="display:inline-block; float:right">
</div>

<div style="text-align:center; clear:both; width:100%; padding-top:15px">
<p style="text-align:center;font-style:italic">(left) The OpenEphys Acquisition board (right) Outline of the PiOperant board without components</p> 
</div>

You can see a general outline of the rig in the following diagram, where the RPiOperant is powered by a Raspberry Pi that runs our [PyOperant python package](https://github.com/gentnerlab/pyoperant) controlling behaviors on a physical panel that the bird interacts with. Neural data is then recorded on the Open Ephys acquisition board, and outputted to a desktop computer via a USB3 cable. 

<div style="text-align:center">
    <img src="assets/img/pioperant-outline.png" style="display:inline-block">
</div>
<div style="text-align:center; clear:both">
      
<p style="text-align:center;font-style:italic">(left) Experimental design with desktop (right) experimental design without.</p> 
</div>

With the updated specs on the raspberry pi however, we can potentially scrap the desktop computer, and run Open Ephys directly on the raspberry pi:

<div style="text-align:center">
      <img src="assets/img/pioperant outline-no-desktop.png" style="display:inline-block">
</div>
<div style="text-align:center; clear:both">
      
<p style="text-align:center;font-style:italic">(left) Experimental design with desktop (right) experimental design without.</p> 
</div>

Here's how it worked: 

<div style="text-align:center">
<div class="centerblock" style="display:inline-block">
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">In case anyone was wondering, yes, you can run <a href="https://twitter.com/OpenEphys?ref_src=twsrc%5Etfw">@OpenEphys</a> on the new <a href="https://twitter.com/Raspberry_Pi?ref_src=twsrc%5Etfw">@Raspberry_Pi</a> 4 <a href="https://t.co/sSgxtDqOn3">pic.twitter.com/sSgxtDqOn3</a></p>&mdash; Tim Sainburg (@tim_sainburg) <a href="https://twitter.com/tim_sainburg/status/1164252464155021314?ref_src=twsrc%5Etfw">August 21, 2019</a></blockquote> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>    
</div>    
</div>

I used the 2GB ram board, but next time would definitely go with the 4GB. 

### How to install Open Ephys on the Raspberry Pi
##### 1) Flash raspian to an SD card and set up your Raspberry Pi

[There are a million tutorials out there on how to set up a raspberry pi, so I won't belabor the point](https://www.raspberrypi.org/documentation/installation/installing-images/). I used Raspian buster with all reccomended software. 


##### 2) Install Open Ephys GUI on the Raspberry Pi

[I followed these instructions, building from source](https://open-ephys.atlassian.net/wiki/spaces/OEW/pages/491546/Linux).
Here you're going to run into a few issues if you follow these directions directly: 
1. ZeroMQ needs to be installed from a raspberry pi repository
2. [You need an ARM version of the frontpanel library](https://github.com/open-ephys/plugin-GUI/issues/314)
3. You need to install the boost library

##### And the fix to each of those things:
 
> **ZeroMQ needs to be installed from a raspberry pi repository**

I think I followed [these instructions](https://github.com/MonsieurV/ZeroMQ-RPi), but you can find the answer to this one with a quick google. 

> **You need an ARM version of the frontpanel library**

As of this posting the ARM version of the frontpanel library is not on the [OpenEphys GUI repository](https://github.com/open-ephys/plugin-GUI). Luckily, the after posting [an issue in the GitHub repo](https://github.com/open-ephys/plugin-GUI/issues/314), I was quickly sent an ARM version of the frontpanel library from the Open Ephys team. If the library isn't put on the repo soon, I would just send a quick email to the open ephys team asking for it! The file will be `libokFrontPanel.so`, and you'll just need to replace the one located in `Builds/Linux/build/libokFrontPanel.so ` before building open-ephys. 

Without the ARM version of frontpanel you will run into this error: 

```
Building RhythmNode.so
/home/pi/github/plugin-GUI/Builds/Linux/build/libokFrontPanel.so: file not recognized: file format not recognized
collect2: error: ld returned 1 exit status
make[1]: *** [Makefile:25: /home/pi/github/plugin-GUI/Builds/Linux/build/plugins/RhythmNode.so] Error 1
make[1]: Leaving directory '/home/pi/github/plugin-GUI/Source/Plugins/RhythmNode'
make: *** [Makefile.plugins:77: ../../Source/Plugins/RhythmNode] Error 2
```

> **You need to install the boost library**

Without the boost library you will run into this error:

```
Loading Plugin: RhythmNode... ../../Source/Processors/PluginManager/PluginManager.cpp:177: Failed to load plugin DLL: libboost_chrono.so.1.62.0: cannot open shared object file: No such file or directory

Loading Plugin: IntanRecordingController... ../../Source/Processors/PluginManager/PluginManager.cpp:177: Failed to load plugin DLL: libboost_chrono.so.1.62.0: cannot open shared object file: No such file or directory
```

The solution is just to install the correct version of Boost: `sudo apt-get install libboost1.62-*`



Thats it! Feel free to reach out if you have any questions. 